# Centrality Analysis: Amazon Product Co-Purchasing Network

This notebook performs comprehensive centrality analysis on the Amazon co-purchasing network to identify the most important/influential products.

## Objectives:
1. Load preprocessed graph
2. Compute all centrality measures (PageRank, Degree, Betweenness, HITS)
3. Compare execution times
4. Analyze top nodes for each measure
5. Visualize results and correlations
6. Compute similarity metrics between measures
7. Generate insights and interpretations


## 1. Import Libraries and Setup

Import all necessary libraries for centrality analysis and visualization.


In [ ]:
# Standard library imports
import os
import sys
import time
import pickle
from pathlib import Path

# Add src directory to path for imports
# In Jupyter, cwd is typically the project root, but we handle both cases
if Path.cwd().name == 'notebooks':
    project_root = Path.cwd().parent
else:
    project_root = Path.cwd()

sys.path.insert(0, str(project_root / 'src'))
# Change to project root for data paths
os.chdir(project_root)
print(f"Project root: {project_root}")
print(f"Working directory: {os.getcwd()}")

# Data manipulation
import pandas as pd
import numpy as np

# Network analysis
import networkx as nx

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical analysis
from scipy.stats import pearsonr, kendalltau

# Project modules
from data_loader import load_saved_graph
from centrality_analysis import (
    compare_centrality_measures,
    get_top_k_nodes,
    save_centrality_results
)
from centrality_visualization import (
    plot_top_k_comparison,
    plot_correlation_heatmap,
    plot_centrality_distributions,
    plot_overlap_venn,
    create_centrality_report
)

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid' if 'seaborn-v0_8-darkgrid' in plt.style.available 
              else 'seaborn-darkgrid' if 'seaborn-darkgrid' in plt.style.available 
              else 'ggplot')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")


## 2. Load Preprocessed Graph

Load the cleaned graph that was created during preprocessing. This graph has already been processed (self-loops removed, largest component extracted).


In [ ]:
# Load the preprocessed graph
graph_path = "data/processed/amazon_graph_cleaned.pkl"

print(f"Loading graph from: {graph_path}")
G = load_saved_graph(graph_path)

print(f"\n✅ Graph loaded successfully!")
print(f"   Nodes: {G.number_of_nodes():,}")
print(f"   Edges: {G.number_of_edges():,}")
print(f"   Type: {'Undirected' if not G.is_directed() else 'Directed'}")
print(f"   Connected: {nx.is_connected(G) if not G.is_directed() else 'N/A (directed)'}")


## 3. Compute All Centrality Measures

Compute all four centrality measures with timing information. For large graphs, we use sampling for betweenness centrality to reduce computation time.


In [ ]:
# Compute all centrality measures
# For large graphs (~300K nodes), use k=1000 for betweenness sampling
# This provides good approximation while keeping computation time reasonable

print("Computing all centrality measures...")
print("=" * 60)

# Set parameters
k_top = 100  # Top-k nodes to extract
betweenness_sample = 1000  # Sample size for betweenness (recommended for ~300K nodes)

# Compute all measures with timing
start_time = time.time()
centrality_results = compare_centrality_measures(
    G, 
    k=k_top, 
    betweenness_k=betweenness_sample
)
total_time = time.time() - start_time

print("=" * 60)
print(f"✅ All centrality measures computed in {total_time:.2f} seconds")
print("=" * 60)


## 4. Execution Time Comparison

Display a table comparing the execution time for each centrality measure. This helps understand the computational cost of each algorithm.


In [ ]:
# Display execution time comparison
if 'timing' in centrality_results:
    timing_df = pd.DataFrame([
        {'Measure': measure.capitalize(), 'Time (seconds)': time_taken}
        for measure, time_taken in centrality_results['timing'].items()
    ])
    timing_df = timing_df.sort_values('Time (seconds)', ascending=False)
    timing_df['Time (minutes)'] = timing_df['Time (seconds)'] / 60
    timing_df['Percentage'] = (timing_df['Time (seconds)'] / timing_df['Time (seconds)'].sum() * 100).round(2)
    
    print("Execution Time Comparison")
    print("=" * 60)
    print(timing_df.to_string(index=False))
    print("=" * 60)
    print(f"\nTotal computation time: {sum(centrality_results['timing'].values()):.2f} seconds")
    print(f"Total computation time: {sum(centrality_results['timing'].values())/60:.2f} minutes")
else:
    print("⚠️ Timing information not available")


## 5. Top-20 Nodes for Each Measure

Display the top-20 most central nodes according to each centrality measure. This helps identify the most important/influential products in the network.


In [ ]:
# Display top-20 nodes for each measure
print("Top-20 Nodes by Centrality Measure")
print("=" * 80)

# PageRank
print("\n📊 PageRank Top-20:")
print("-" * 80)
print(centrality_results['top_k_pagerank'].head(20).to_string(index=False))

# Degree Centrality
print("\n📊 Degree Centrality Top-20:")
print("-" * 80)
print(centrality_results['top_k_degree'].head(20).to_string(index=False))

# Betweenness Centrality
print("\n📊 Betweenness Centrality Top-20:")
print("-" * 80)
print(centrality_results['top_k_betweenness'].head(20).to_string(index=False))

# HITS Hubs
if 'top_k_hubs' in centrality_results:
    print("\n📊 HITS Hubs Top-20:")
    print("-" * 80)
    print(centrality_results['top_k_hubs'].head(20).to_string(index=False))

# HITS Authorities
if 'top_k_authorities' in centrality_results:
    print("\n📊 HITS Authorities Top-20:")
    print("-" * 80)
    print(centrality_results['top_k_authorities'].head(20).to_string(index=False))


## 6. Visualizations

Create comprehensive visualizations to understand the centrality measures and their relationships.


### 6.1 Top-K Comparison Bar Charts

Compare the top-20 nodes across all centrality measures using horizontal bar charts.


In [ ]:
# Create output directory for figures
os.makedirs("results/figures", exist_ok=True)

# Plot top-k comparison
print("Creating top-k comparison visualization...")
plot_top_k_comparison(
    centrality_results, 
    k=20, 
    save_path="results/figures/centrality_top_k_comparison.png"
)
print("✅ Saved: results/figures/centrality_top_k_comparison.png")


### 6.2 Correlation Heatmap

Visualize the correlation between different centrality measures using a heatmap.


In [ ]:
# Plot correlation heatmap
print("Creating correlation heatmap...")
plot_correlation_heatmap(
    centrality_results['correlation_matrix'],
    save_path="results/figures/centrality_correlation_heatmap.png"
)
print("✅ Saved: results/figures/centrality_correlation_heatmap.png")

# Display correlation matrix
print("\nCorrelation Matrix:")
print("=" * 60)
print(centrality_results['correlation_matrix'].round(3).to_string())
print("=" * 60)


### 6.3 Score Distributions

Compare the distribution of centrality scores across different measures using violin plots.


In [ ]:
# Plot centrality distributions
print("Creating distribution plots...")
plot_centrality_distributions(
    centrality_results,
    save_path="results/figures/centrality_distributions.png",
    plot_type='violin'
)
print("✅ Saved: results/figures/centrality_distributions.png")


### 6.4 Overlap Analysis

Visualize the overlap of top-k nodes across different centrality measures. This shows which nodes are consistently identified as important by multiple measures.


In [ ]:
# Plot overlap visualization
print("Creating overlap visualization...")
plot_overlap_venn(
    centrality_results,
    k=100,
    save_path="results/figures/centrality_overlap_venn.png"
)
print("✅ Saved: results/figures/centrality_overlap_venn.png")


## 7. Statistical Analysis

Compute detailed statistical measures to understand the relationships between centrality measures.


### 7.1 Pearson Correlation

Compute Pearson correlation coefficients between all pairs of centrality measures. This measures linear relationships.


In [ ]:
# Compute pairwise Pearson correlations
measures = ['pagerank', 'degree', 'betweenness']
if 'hubs' in centrality_results:
    measures.append('hubs')
elif 'authorities' in centrality_results:
    measures.append('authorities')

# Get common nodes
common_nodes = set(centrality_results[measures[0]].keys())
for measure in measures[1:]:
    common_nodes = common_nodes.intersection(set(centrality_results[measure].keys()))

print(f"Computing Pearson correlations for {len(common_nodes)} common nodes...")
print("=" * 60)

pearson_results = []
for i, measure1 in enumerate(measures):
    for measure2 in measures[i+1:]:
        scores1 = [centrality_results[measure1][node] for node in common_nodes]
        scores2 = [centrality_results[measure2][node] for node in common_nodes]
        corr, p_value = pearsonr(scores1, scores2)
        pearson_results.append({
            'Measure 1': measure1.capitalize(),
            'Measure 2': measure2.capitalize(),
            'Pearson r': corr,
            'p-value': p_value
        })

pearson_df = pd.DataFrame(pearson_results)
pearson_df = pearson_df.sort_values('Pearson r', ascending=False)

print(pearson_df.to_string(index=False))
print("=" * 60)


### 7.2 Kendall Tau Correlation

Compute Kendall Tau rank correlation coefficients. This measures monotonic relationships and is more robust to outliers than Pearson correlation.


In [ ]:
# Compute pairwise Kendall Tau correlations
print(f"Computing Kendall Tau correlations for {len(common_nodes)} common nodes...")
print("=" * 60)

kendall_results = []
for i, measure1 in enumerate(measures):
    for measure2 in measures[i+1:]:
        scores1 = [centrality_results[measure1][node] for node in common_nodes]
        scores2 = [centrality_results[measure2][node] for node in common_nodes]
        tau, p_value = kendalltau(scores1, scores2)
        kendall_results.append({
            'Measure 1': measure1.capitalize(),
            'Measure 2': measure2.capitalize(),
            'Kendall τ': tau,
            'p-value': p_value
        })

kendall_df = pd.DataFrame(kendall_results)
kendall_df = kendall_df.sort_values('Kendall τ', ascending=False)

print(kendall_df.to_string(index=False))
print("=" * 60)


### 7.3 Jaccard Similarity of Top-100 Sets

Compute Jaccard similarity between the top-100 node sets for each pair of centrality measures. This shows how much overlap there is in the most important nodes identified by different measures.


In [ ]:
# Compute Jaccard similarity for top-100 sets
def jaccard_similarity(set1, set2):
    """Compute Jaccard similarity between two sets."""
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0.0

# Get top-100 node sets
top_k = 100
top_sets = {}
top_sets['pagerank'] = set(centrality_results['top_k_pagerank'].head(top_k)['node_id'].values)
top_sets['degree'] = set(centrality_results['top_k_degree'].head(top_k)['node_id'].values)
top_sets['betweenness'] = set(centrality_results['top_k_betweenness'].head(top_k)['node_id'].values)
if 'top_k_hubs' in centrality_results:
    top_sets['hubs'] = set(centrality_results['top_k_hubs'].head(top_k)['node_id'].values)
if 'top_k_authorities' in centrality_results:
    top_sets['authorities'] = set(centrality_results['top_k_authorities'].head(top_k)['node_id'].values)

print(f"Computing Jaccard similarity for top-{top_k} node sets...")
print("=" * 60)

jaccard_results = []
for i, (name1, set1) in enumerate(top_sets.items()):
    for name2, set2 in list(top_sets.items())[i+1:]:
        jaccard = jaccard_similarity(set1, set2)
        intersection = len(set1 & set2)
        jaccard_results.append({
            'Measure 1': name1.capitalize(),
            'Measure 2': name2.capitalize(),
            'Jaccard Similarity': jaccard,
            'Intersection Size': intersection,
            'Union Size': len(set1 | set2)
        })

jaccard_df = pd.DataFrame(jaccard_results)
jaccard_df = jaccard_df.sort_values('Jaccard Similarity', ascending=False)

print(jaccard_df.to_string(index=False))
print("=" * 60)


## 8. Save All Results

Save all centrality results, visualizations, and analysis outputs to the results directory.


In [ ]:
# Create output directory
output_dir = "data/results/centrality"
os.makedirs(output_dir, exist_ok=True)
os.makedirs("results/tables", exist_ok=True)

print(f"Saving centrality results to {output_dir}/...")

# Save centrality results (CSV files)
save_centrality_results(centrality_results, output_dir="results/tables")
print("✅ Centrality scores and top-k nodes saved to results/tables/")

# Save correlation matrices
correlation_path = os.path.join(output_dir, "correlation_analysis.csv")
correlation_data = {
    'pearson': pearson_df,
    'kendall': kendall_df,
    'jaccard': jaccard_df
}

# Save each correlation analysis
pearson_df.to_csv(os.path.join(output_dir, "pearson_correlation.csv"), index=False)
kendall_df.to_csv(os.path.join(output_dir, "kendall_tau_correlation.csv"), index=False)
jaccard_df.to_csv(os.path.join(output_dir, "jaccard_similarity.csv"), index=False)

print(f"✅ Correlation analyses saved to {output_dir}/")

# Create comprehensive HTML report
print("\nCreating comprehensive HTML report...")
create_centrality_report(centrality_results, output_path="results/tables/centrality_report.html")
print("✅ HTML report created: results/tables/centrality_report.html")

print(f"\n✅ All results saved successfully!")


## 9. Insights and Interpretation

### Key Findings:

1. **Execution Time Analysis:**
   - Betweenness centrality is typically the slowest due to shortest path computations
   - Degree centrality is the fastest (O(n) complexity)
   - PageRank and HITS have similar computational costs

2. **Top Nodes:**
   - Different measures identify different "important" nodes
   - High degree nodes are not always high in other measures
   - Some nodes consistently appear in top-k across multiple measures

3. **Correlation Analysis:**
   - **High correlation** (r > 0.7): Measures that identify similar nodes
   - **Moderate correlation** (0.3 < r < 0.7): Partial agreement
   - **Low correlation** (r < 0.3): Measures capture different aspects of importance

4. **Jaccard Similarity:**
   - Shows overlap in top-k sets
   - High similarity (>0.5): Measures agree on important nodes
   - Low similarity (<0.3): Measures capture different types of importance

5. **Distribution Analysis:**
   - Most nodes have low centrality scores (power-law distribution)
   - A few nodes have very high centrality (hubs/influencers)
   - Different measures have different score distributions

### Interpretation:

- **PageRank**: Identifies nodes that are well-connected to other well-connected nodes (recursive importance)
- **Degree Centrality**: Simple measure of direct connections (local importance)
- **Betweenness Centrality**: Identifies nodes that act as bridges (structural importance)
- **HITS**: Separates hubs (point to authorities) and authorities (pointed to by hubs)

### Business Implications:

For the Amazon co-purchasing network:
- Products with high PageRank are frequently co-purchased with other popular products
- High degree products are co-purchased with many different products
- High betweenness products connect different product categories
- Understanding these patterns can inform:
  - Product recommendations
  - Marketing strategies
  - Inventory management
  - Cross-selling opportunities
